### Set up library and download file GGUF

In [15]:
!pip install llama-cpp-python

In [16]:
!pip install llama-cpp-python[server]

In [17]:
import requests
import os

url = 'https://huggingface.co/QuantFactory/gemma-2-9b-GGUF/resolve/main/gemma-2-9b.Q4_K_M.gguf?download=true'
save_directory = 'models'
filename = 'gemma-2-9b.Q4_K_M.gguf'

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

file_path = os.path.join(save_directory, filename)

if os.path.exists(file_path):
    print(f'Tệp {filename} đã tồn tại trong thư mục {save_directory}.')
else:
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))  # Total size in bytes
        block_size = 8192  # Download in chunks of 8KB
        progress = 0  # Track the progress

        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    file.write(chunk)
                    progress += len(chunk)
                    percent_complete = (progress / total_size) * 100
                    print(f'Tải xuống: {percent_complete:.2f}%', end='\r')

        print(f'\nTệp đã được tải xuống và lưu tại {file_path}')
    else:
        print('Có lỗi xảy ra trong quá trình tải xuống.')


Tệp gemma-2-9b.Q4_K_M.gguf đã tồn tại trong thư mục models.


### Run Llama.cpp server
**python -m llama_cpp.server --model models/PhoGPT-4B-Chat-Q4_K_M.gguf --n_ctx 8192**

### Call model via API (llama_cpp server) with Langchain

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(base_url="http://localhost:8000/v1", api_key="llama.cpp")

### Define the Offline_RAG Class

In [20]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


class Offline_RAG:
    def __init__(self, llm) -> None:
        self.store = {}
        self.llm = llm
        self.messages = [
            (
                "user","Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. Dựa vào ngữ cảnh bên dưới hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi của họ. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời.\n Ngữ cảnh: \n{context}\n\n Câu hỏi: {question}"
            )
        ]
        self.prompt = ChatPromptTemplate.from_messages(self.messages)
        self.with_message_history = RunnableWithMessageHistory(llm, self.get_session_history)

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = InMemoryChatMessageHistory()
        return self.store[session_id]

    def get_chain(self, retriever):
        input_data = {
            "context": retriever | self.format_docs,
            "question": RunnablePassthrough()
        }
        rag_chain = (
            input_data
            | self.prompt
            #| self.with_message_history
        )
        return rag_chain
    
    def format_docs(self, docs):
        formatted_docs = "\n\n".join(doc.page_content for doc in docs)
        self.list_doc ={"context" :formatted_docs}
        print(self.list_doc)
        return formatted_docs

### Define the VectorDB class using Pinecone

In [21]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

class VectorDB:
    def __init__(self,
                 embedding = HuggingFaceEmbeddings()) -> None:
        os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
        self.pinecone_api_key = os.getenv('PINECONE_API_KEY')
        self.embedding = embedding
        self.db = self._build_db()
    
    def _build_db(self):
        db = PineconeVectorStore.from_existing_index(
            index_name="docs-rag-chatbot",
            namespace="docs-search",
            embedding=self.embedding
        )
        return db
    
    # tính similarity
    def get_retriever(self,
                      search_type: str = "similarity",
                      search_kwargs: dict = {"k": 5}
                      ):
        retriever =self.db.as_retriever(search_type=search_type,
                                        search_kwargs=search_kwargs)
        return retriever

c:\Users\Admin\anaconda3\envs\chatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
c:\Users\Admin\anaconda3\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Admin\anaconda3\envs\chatbot\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set

### Function build chain

In [22]:
def build_rag_chain(llm):
    retriever = VectorDB().get_retriever()
    off_rag = Offline_RAG(llm)
    rag_chain = off_rag.get_chain(retriever)
    return rag_chain, off_rag

def build_rag_chain_from_off_rag(off_rag):
    retriever = VectorDB().get_retriever()
    rag_chain = off_rag.get_chain(retriever)
    return rag_chain

### Run example

In [23]:
config = {"configurable": {"session_id": "ss1"}}
chain, off_rag = build_rag_chain(llm)
while True:
    question = input("You: ")
    if question == "exit":
        break
    message = chain.invoke(question)
    print(message.messages)
    answer = off_rag.with_message_history.invoke(message.messages, config=config)
    print(off_rag.store)
    print(answer.content)

{'context': 'lời của em sẽ được giữ bí mật, và em không cần phải trả lời bất kỳ câu hỏi mà em không muốn. Tôi \ncó thể bắt đầu hỏi Em?\n\nlưu ý giúp phụ nữ ngăn ngừa trầm cảm sau sinhTham gia khóa học tiền sảnYêu cầu giúp đỡ từ người thânKhông quá áp lực việc chăm conTrầm cảm sau sinh là gì?\n\nĐối với người con có mẹ mắc bệnh trầm cảm\xa0\nNhững đứa trẻ có mẹ bị trầm cảm nhiều nguy cơ không phát triển cảm xúc, hành vi:\n\nChậm phát triển ngôn ngữ, vận động;\nHạn chế khả năng giao tiếp;\nCó thể có những hành vi bất thường hoặc dễ kích động hơn trẻ bình thường;\nTrẻ dễ căng thẳng và khó thích nghi với môi trường, khó hòa nhập xã hội…\n\nĐối với gia đình\n\ngiúp đỡ của mọi người. \nTất cả các thông tin mà chị cung cấp là tự nguyện, và sẽ được giữ bí mật \nnghiêm ngặt và chỉ phục vụ cho mục đích nghiên cứu. Chị có thể dừng cuộc phỏng \nvấn tại bất kỳ điểm nào, hoặc không trả lời bất cứ câu hỏi nào mà chúng tôi yêu \ncầu. Tôi sẽ không viết ra tên của chị. Câu trả lời của chị sẽ chỉ được sử